In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
import csv
import string
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image

# =========================
# CONFIGURATION
# =========================
DATA_DIR = '/content/drive/MyDrive/archive (5)/Images'
CAPTION_FILE = '/content/drive/MyDrive/archive (5)/captions.txt'

VOCAB_SIZE = 10000
MAX_LENGTH = 40
EMBEDDING_DIM = 256
LSTM_UNITS = 256
BATCH_SIZE = 64
EPOCHS = 20

# =========================
# LOAD & CLEAN CAPTIONS
# =========================


def load_captions(caption_file):
    captions_dict = defaultdict(list)
    with open(caption_file, 'r') as f:
        reader = csv.DictReader(f)  # Handles the header: image,caption
        for row in reader:
            img_id = row['image'].strip()
            caption = row['caption'].strip().lower()
            caption = caption.translate(str.maketrans('', '', string.punctuation))
            caption = 'startseq ' + caption + ' endseq'
            captions_dict[img_id].append(caption)
    return captions_dict

# Debug: See how many images match
all_captioned_images = set(captions_dict.keys())
existing_images = set(os.listdir(DATA_DIR))
matched = all_captioned_images & existing_images

print(f"Total images in captions.txt: {len(all_captioned_images)}")
print(f"Total images in folder: {len(existing_images)}")
print(f"Matched image-caption pairs: {len(matched)}")


# =========================
# EXTRACT FEATURES
# =========================
def extract_image_features(image_dir, image_list):
    model = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
    features = {}
    for img_name in tqdm(image_list):
        img_path = os.path.join(image_dir, img_name)
        if not os.path.exists(img_path):
            continue
        try:
            img = image.load_img(img_path, target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            feature = model.predict(img_array, verbose=0)
            features[img_name] = feature
        except Exception as e:
            print(f"Error with {img_name}: {e}")
    return features

# =========================
# TOKENIZER
# =========================
def create_tokenizer(captions, vocab_size):
    tokenizer = Tokenizer(num_words=vocab_size, oov_token='<unk>', filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(captions)
    return tokenizer

# =========================
# DATA GENERATOR
# =========================
def data_generator(image_features, captions_dict, tokenizer, max_length, vocab_size):
    for img_id, captions in captions_dict.items():
        if img_id not in image_features:
            continue
        for caption in captions:
            seq = tokenizer.texts_to_sequences([caption])[0]
            for i in range(1, len(seq)):
                in_seq, out_seq = seq[:i], seq[i]
                in_seq = pad_sequences([in_seq], maxlen=max_length, padding='post')[0]
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                img_feature = image_features[img_id][0]
                yield (img_feature, in_seq), out_seq

# =========================
# BUILD MODEL
# =========================
def build_decoder(vocab_size, max_length, embedding_dim, units):
    inputs1 = Input(shape=(1280,))
    fe1 = Dropout(0.4)(inputs1)
    fe2 = Dense(units, activation='relu')(fe1)

    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
    se2 = Dropout(0.4)(se1)
    se3 = LSTM(units)(se2)

    decoder1 = add([fe2, se3])
    decoder2 = Dense(units, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)

    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    return model

# =========================
# GENERATE CAPTION
# =========================
def generate_caption(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
        yhat = model.predict([photo, sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = tokenizer.index_word.get(yhat)
        if word is None or word == 'endseq':
            break
        in_text += ' ' + word
    return in_text.replace('startseq', '').strip()

# =========================
# MAIN
# =========================
if __name__ == "__main__":
    print("Loading captions...")
    captions_dict = load_captions(CAPTION_FILE)

    image_list = [img for img in captions_dict.keys() if os.path.exists(os.path.join(DATA_DIR, img))]
    print(f"Number of valid images: {len(image_list)}")

    print("Extracting image features...")
    image_features = extract_image_features(DATA_DIR, image_list)

    print("Preparing tokenizer...")
    all_captions = [caption for caps in captions_dict.values() for caption in caps]
    tokenizer = create_tokenizer(all_captions, VOCAB_SIZE)

    print("Building model...")
    model = build_decoder(VOCAB_SIZE, MAX_LENGTH, EMBEDDING_DIM, LSTM_UNITS)
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    print("Preparing dataset...")
    total_captions = sum(len(v) for v in captions_dict.values())
    steps_per_epoch = total_captions // BATCH_SIZE
    print(f"Total captions: {total_captions}, Steps per epoch: {steps_per_epoch}")

    if steps_per_epoch == 0:
        raise ValueError("Not enough data to train. Check if images and captions match.")

    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(image_features, captions_dict, tokenizer, MAX_LENGTH, VOCAB_SIZE),
        output_signature=(
            (
                tf.TensorSpec(shape=(1280,), dtype=tf.float32),
                tf.TensorSpec(shape=(MAX_LENGTH,), dtype=tf.int32)
            ),
            tf.TensorSpec(shape=(VOCAB_SIZE,), dtype=tf.float32)
        )
    ).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    print("Training model...")
    model.fit(dataset, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)

    print("\nGenerating caption for test image...")
    test_img = list(image_features.keys())[0]
    test_feat = image_features[test_img]
    caption = generate_caption(model, tokenizer, test_feat, MAX_LENGTH)

    img_path = os.path.join(DATA_DIR, test_img)
    img = Image.open(img_path)

    plt.imshow(img)
    plt.axis('off')
    plt.title(caption)
    plt.show()
pip install nltk rouge-score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
def evaluate_model(model, captions_dict, image_features, tokenizer, max_length):
    from nltk.translate.bleu_score import corpus_bleu
    from nltk.translate.meteor_score import meteor_score
    from rouge_score import rouge_scorer
    import numpy as np
    from tqdm import tqdm

    references = []
    hypotheses = []

    rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    meteor_scores, rouge1_scores, rougeL_scores = [], [], []

    for img_id, refs in tqdm(captions_dict.items(), desc="Evaluating"):
        if img_id not in image_features:
            continue
        photo = image_features[img_id]
        pred = generate_caption(model, tokenizer, photo, max_length)

        tokenized_refs = [r.split() for r in refs]
        tokenized_pred = pred.split()

        references.append(tokenized_refs)
        hypotheses.append(tokenized_pred)

        meteor_scores.append(meteor_score(tokenized_refs, tokenized_pred))
        rouge_scores = rouge.score(refs[0], pred)
        rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
        rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

    print("\nEvaluation Metrics:")
    print("BLEU-1:", corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0)))
    print("BLEU-2:", corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0, 0)))
    print("BLEU-3:", corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0)))
    print("BLEU-4:", corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25)))
    print("METEOR:", np.mean(meteor_scores))
    print("ROUGE-1:", np.mean(rouge1_scores))
    print("ROUGE-L:", np.mean(rougeL_scores))
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but often needed
evaluate_model(model, captions_dict, image_features, tokenizer, MAX_LENGTH)
pip install streamlit
!pip install streamlit
!pip install pyngrok
!pip install streamlit pyngrok tensorflow pillow
!pip install gradio
!pip install gradio
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# Load the pretrained BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Caption generation function
def generate_caption(image):
    inputs = processor(images=image, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

# Gradio interface
iface = gr.Interface(
    fn=generate_caption,
    inputs=gr.Image(type="pil"),
    outputs=gr.Textbox(),
)

iface.launch(share=True)